<a href="https://colab.research.google.com/github/Rishii-jain/Machine-Learning-Algorithms/blob/main/Product_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/Nike_UK_2022-09-01.csv')

<ipython-input-1-e0e491f2c632>:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/Nike_UK_2022-09-01.csv')


In [2]:
df.columns

Index(['DEPARTMENT', 'CATEGORY', 'SUBCATEGORY', 'SKU', 'SKU_VARIANT',
       'PRODUCT_NAME', 'PRODUCT_ID', 'TITLE', 'PRODUCT_TYPE', 'PRODUCT_URL',
       'PRODUCT_SIZE', 'LABEL', 'IS_BESTSELLER', 'COLOR', 'BRAND',
       'AVAILABILITY', 'CURRENCY', 'PRICE_CURRENT', 'PRICE_RETAIL', 'RunDate',
       'InsertUpdateTime', 'COLOR_CODE', 'TID'],
      dtype='object')

In [3]:
df.drop(['SKU','SKU_VARIANT', 'PRODUCT_ID', 'PRODUCT_URL', 'COLOR', 'AVAILABILITY', 'CURRENCY','PRICE_CURRENT', 'PRICE_RETAIL', 'RunDate', 'InsertUpdateTime', 'COLOR_CODE', 'TID', 'PRODUCT_SIZE', 'IS_BESTSELLER','LABEL'], axis=1, errors='ignore', inplace=True)

In [4]:
df.isnull().sum()

,0
DEPARTMENT,0
CATEGORY,0
SUBCATEGORY,0
PRODUCT_NAME,0
TITLE,0
PRODUCT_TYPE,0
BRAND,36


In [7]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, hamming_loss

In [8]:
# Reduce the dataset to 1000 rows by random sampling
df_sampled = df.sample(n=1000, random_state=42).reset_index(drop=True)

In [9]:
# Combine PRODUCT_NAME and TITLE for text input
df['combined_text'] = df['PRODUCT_NAME'] + " " + df['TITLE']
# Drop rows where combined_text is NaN
df = df.dropna(subset=['combined_text']).reset_index(drop=True)

# Confirm there are no NaN values in the combined_text column
print(df['combined_text'].isnull().sum())


# Text vectorization using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['combined_text'])


0


In [ ]:
# List of columns to check for null values
columns_with_nulls = ['TITLE', 'PRODUCT_TYPE', 'BRAND', 'combined_text']

# Drop rows with null values in the specified columns
df_cleaned = df.dropna(subset=columns_with_nulls).reset_index(drop=True)

In [11]:
# Encode target columns using LabelEncoder
label_encoders = {}
y_encoded = pd.DataFrame()

for column in ['DEPARTMENT', 'CATEGORY', 'SUBCATEGORY', 'PRODUCT_TYPE', 'BRAND']:
    le = LabelEncoder()
    y_encoded[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)


In [13]:
# Initialize Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Wrap with MultiOutputClassifier
model = MultiOutputClassifier(rf)

# Train the model
model.fit(X_train, y_train)


MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

In [15]:
# Make predictions
y_pred = model.predict(X_test)

# Classification Report for Each Target Column
for i, column in enumerate(y_encoded.columns):
    print(f"Classification Report for {column}:\n")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))


Classification Report for DEPARTMENT:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9012
           1       1.00      1.00      1.00      6313

    accuracy                           1.00     15325
   macro avg       1.00      1.00      1.00     15325
weighted avg       1.00      1.00      1.00     15325

Classification Report for CATEGORY:

              precision    recall  f1-score   support

           0       0.94      0.68      0.79       276
           1       0.99      1.00      1.00     10712
           2       1.00      1.00      1.00      4337

    accuracy                           0.99     15325
   macro avg       0.98      0.89      0.93     15325
weighted avg       0.99      0.99      0.99     15325

Classification Report for SUBCATEGORY:

              precision    recall  f1-score   support

           0       0.59      0.56      0.57       231
           1       0.35      0.34      0.35      4048
           2 

# **XGBoost**

In [17]:
!pip install xgboost
from xgboost import XGBClassifier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 5.4 MB/s eta 0:00:00


In [19]:
# Initialize XGBoost Classifier
# For multi:softmax, num_class needs to be explicitly set to the number of unique classes in each target column
num_classes = [len(y_train[col].unique()) for col in y_train.columns]

xgb = XGBClassifier(objective='multi:softmax',
                    eval_metric='mlogloss',
                    use_label_encoder=False,
                    n_estimators=100,
                    num_class=num_classes[0]  # Assuming all target columns have the same number of classes
                    )

# Wrap with MultiOutputClassifier
model = MultiOutputClassifier(xgb)

# Train the model
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:02:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:02:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:02:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:02:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:02:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='mlogloss',
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=100, n_jobs=None,
                                              num_class=2,
                                              num_parallel_tree=None, ...))

In [20]:
# Make predictions
y_pred = model.predict(X_test)

# Classification Report for Each Target Column
for i, column in enumerate(y_encoded.columns):
    print(f"Classification Report for {column}:\n")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

# Convert y_test and y_pred to binary format for Hamming Loss
y_test_binary = pd.get_dummies(y_test.stack()).stack().to_numpy().reshape(y_test.shape[0], -1)
y_pred_binary = pd.get_dummies(pd.DataFrame(y_pred).stack()).stack().to_numpy().reshape(y_test.shape[0], -1)

# Hamming Loss
print("Hamming Loss:", hamming_loss(y_test_binary, y_pred_binary))


Classification Report for DEPARTMENT:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9012
           1       1.00      1.00      1.00      6313

    accuracy                           1.00     15325
   macro avg       1.00      1.00      1.00     15325
weighted avg       1.00      1.00      1.00     15325

Classification Report for CATEGORY:

              precision    recall  f1-score   support

           0       0.94      0.70      0.80       276
           1       0.99      1.00      1.00     10712
           2       1.00      1.00      1.00      4337

    accuracy                           0.99     15325
   macro avg       0.98      0.90      0.93     15325
weighted avg       0.99      0.99      0.99     15325

Classification Report for SUBCATEGORY:

              precision    recall  f1-score   support

           0       0.61      0.57      0.59       231
           1       0.38      0.32      0.35      4048
           2 

# **lightGBM**

In [26]:
!pip install lightgbm
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier

# Get the number of classes for the first target column
# (Assuming all target columns have the same number of classes)
num_classes = len(y_train[y_train.columns[0]].unique())

# Initialize a single LightGBM Classifier with num_class
estimator = LGBMClassifier(objective='multiclass',
                            boosting_type='gbdt',
                            n_estimators=200,
                            num_leaves=31,
                            num_class=num_classes)

# Wrap with MultiOutputClassifier using the single estimator
model = MultiOutputClassifier(estimator)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
for i, column in enumerate(y_encoded.columns):
    print(f"Classification Report for {column}:\n")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [27]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Convert targets to categorical (one-hot encoded)
y_categorical = [to_categorical(y_encoded[col]) for col in y_encoded.columns]

# Define the input shape based on TF-IDF features
input_layer = Input(shape=(X_train.shape[1],))

# Shared dense layers for feature extraction
x = Dense(128, activation='relu')(input_layer)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)

# Separate output layers for each target column
outputs = []
for col in y_categorical:
    outputs.append(Dense(col.shape[1], activation='softmax')(x))

# Define the model
model = Model(inputs=input_layer, outputs=outputs)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train.toarray(),
          [col[y_train.index] for col in y_categorical],
          validation_split=0.2,
          epochs=50,
          batch_size=32,
          callbacks=[early_stopping])

# Make predictions
y_pred = model.predict(X_test.toarray())

# Decode predictions and evaluate
decoded_predictions = {
    column: label_encoders[column].inverse_transform(y_pred[i].argmax(axis=1))
    for i, column in enumerate(y_encoded.columns)
}

# Evaluation: Classification Report for Each Target
for i, column in enumerate(y_encoded.columns):
    print(f"Classification Report for {column}:\n")
    print(classification_report(y_test.iloc[:, i], y_pred[i].argmax(axis=1)))




Epoch 1/50
1533/1533 [==============================] - 9s 5ms/step - loss: 1.8441 - dense_2_loss: 0.0497 - dense_3_loss: 0.0659 - dense_4_loss: 1.3115 - dense_5_loss: 0.0631 - dense_6_loss: 0.3539 - dense_2_accuracy: 0.9810 - dense_3_accuracy: 0.9778 - dense_4_accuracy: 0.4594 - dense_5_accuracy: 0.9794 - dense_6_accuracy: 0.8752 - val_loss: 1.1870 - val_dense_2_loss: 0.0055 - val_dense_3_loss: 0.0171 - val_dense_4_loss: 0.9901 - val_dense_5_loss: 0.0052 - val_dense_6_loss: 0.1691 - val_dense_2_accuracy: 0.9980 - val_dense_3_accuracy: 0.9931 - val_dense_4_accuracy: 0.4978 - val_dense_5_accuracy: 0.9988 - val_dense_6_accuracy: 0.9281
Epoch 2/50
1533/1533 [==============================] - 6s 4ms/step - loss: 1.1584 - dense_2_loss: 0.0077 - dense_3_loss: 0.0153 - dense_4_loss: 0.9671 - dense_5_loss: 0.0033 - dense_6_loss: 0.1651 - dense_2_accuracy: 0.9977 - dense_3_accuracy: 0.9942 - dense_4_accuracy: 0.5030 - dense_5_accuracy: 0.9994 - dense_6_accuracy: 0.9304 - val_loss: 1.1201 - val_